Setting Directory

In [1]:
import os
from pathlib import Path

In [2]:
import logging

In [3]:
%pwd

'/Volumes/T7/DL_Skin_Cancer_Project/skin_cancer_diagnosis/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Volumes/T7/DL_Skin_Cancer_Project/skin_cancer_diagnosis'

Training Component

Import Configurations

In [6]:
from skinCancerDiagnosis.entity.config_entity import TrailTrainingConfig
from skinCancerDiagnosis.config.configuration import ConfugarationManager
from skinCancerDiagnosis.components.data_prep import DataGenerator

Import Nesessary Libraries

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

Training Component

In [9]:
class vgg19Training:
    def __init__(self, config:TrailTrainingConfig):
        self.config = config
        self.early_stopping = EarlyStopping(monitor='val_loss',patience=config.params_patience,restore_best_weights=True)

    def define_base_model(self):
        base_model = VGG19(weights=self.config.params_weights, include_top=self.config.params_include_top)

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        predictions = Dense(8, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        for layer in base_model.layers:
            layer.trainable = False

        logging.info(model.summary())
        
        return model
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model_path = os.path.join(path,'vgg19.h5')
        model.save(model_path)
    
    def get_model_and_compile(self):
        with tf.device('/GPU:0'):
            self.vgg19_model = self.define_base_model()
            self.vgg19_model.compile(
                optimizer = Adam(learning_rate=self.config.params_learning_rate),
                loss = "categorical_crossentropy",
                metrics = ['accuracy']
            )
    
    def train_model(self,train_generator,val_generator):
        with tf.device('/GPU:0'):
            self.history = self.vgg19_model.fit(
                train_generator,
                steps_per_epoch = len(train_generator),
                epochs = self.config.params_epochs,
                validation_data = val_generator,
                validation_steps = len(val_generator),
                callbacks = [self.early_stopping]
            )

        self.save_model(
            path=self.config.root_dir,
            model=self.vgg19_model
        )



Training Pipeline

In [10]:
try:
    config = ConfugarationManager()

    data_prep_config = config.get_data_prep_config()
    generator = DataGenerator(config=data_prep_config)
    train_generator = generator.get_train_generator()
    val_generator = generator.get_val_generator()
    
    trail_train_config = config.get_trail_training_config()
    vgg_trainer = vgg19Training(config=trail_train_config)
    vgg_trainer.get_model_and_compile()
    vgg_trainer.train_model(train_generator=train_generator,val_generator=val_generator)

except Exception as e:
    raise e

[2024-04-20 11:42:18,194: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-20 11:42:18,196: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-20 11:42:18,197: INFO: common: created directory at: artifacts]
Found 21491 images belonging to 8 classes.
Found 1910 images belonging to 8 classes.
[2024-04-20 11:42:20,134: INFO: common: created directory at: artifacts/trail_training]


2024-04-20 11:42:20.144951: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-04-20 11:42:20.145003: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-04-20 11:42:20.145019: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-04-20 11:42:20.145611: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-20 11:42:20.145659: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 20,557,896 (78.42 MB)

...

2024-04-20 14:16:45.597167: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-20 14:16:45.597197: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11749444730991379000
2024-04-20 14:16:45.597201: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 518402207504589776
2024-04-20 14:16:45.597205: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6671623724907996558
2024-04-20 14:16:45.597211: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 13641577357546919972
2024-04-20 14:16:45.597223: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14601439287949195720
2024-04-20 14:16:45.597231: W tensorflow/core/framework/local_rendezv

672/672 ━━━━━━━━━━━━━━━━━━━━ 875s 1s/step - accuracy: 0.6177 - loss: 1.0691 - val_accuracy: 0.5827 - val_loss: 1.1485
Epoch 20/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 153us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 21/100


2024-04-20 14:31:20.357339: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-20 14:31:20.357366: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11749444730991379000
2024-04-20 14:31:20.357370: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 518402207504589776
2024-04-20 14:31:20.357373: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6671623724907996558
2024-04-20 14:31:20.357376: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 13641577357546919972
2024-04-20 14:31:20.357383: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14601439287949195720
2024-04-20 14:31:20.357419: I tensorflow/core/framework/local_rendezv

672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6187 - loss: 1.0604

KeyboardInterrupt: 